In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
%matplotlib inline

In [2]:
pd.set_option('max_columns', None)
df_train = pd.read_csv('train_2.csv', low_memory=False, index_col='Unnamed: 0')
df_train.shape

(225000, 34)

In [3]:
df_test = pd.read_csv('test_2.csv', index_col='Unnamed: 0')
df_test.shape

(25305, 27)

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225000 entries, 0 to 224999
Data columns (total 34 columns):
ticket_id                     225000 non-null int64
agency_name                   225000 non-null object
inspector_name                225000 non-null object
violator_name                 224969 non-null object
violation_street_number       225000 non-null float64
violation_street_name         225000 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    222105 non-null float64
mailing_address_str_name      224996 non-null object
city                          225000 non-null object
state                         224979 non-null object
zip_code                      224999 non-null object
non_us_str_code               1 non-null object
country                       225000 non-null object
ticket_issued_date            225000 non-null object
hearing_date                  214104 non-null object
violation_code                225000 non-null obj

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25305 entries, 225001 to 250305
Data columns (total 27 columns):
ticket_id                     25305 non-null int64
agency_name                   25305 non-null object
inspector_name                25305 non-null object
violator_name                 25302 non-null object
violation_street_number       25305 non-null float64
violation_street_name         25305 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    24598 non-null float64
mailing_address_str_name      25305 non-null object
city                          25305 non-null object
state                         25233 non-null object
zip_code                      25305 non-null object
non_us_str_code               2 non-null object
country                       25305 non-null object
ticket_issued_date            25305 non-null object
hearing_date                  23710 non-null object
violation_code                25305 non-null object
violati

In [3]:
rows_with_missing = df_train[df_train['compliance'].isnull()].index

In [4]:
columns_drop = ['violation_zip_code', 'non_us_str_code', 'grafitti_status', 'compliance_detail', 
                'payment_amount', 'balance_due', 'payment_date', 'payment_status', 'collection_status']
df_train = df_train.drop(columns_drop, axis=1)
df_train = df_train.drop(rows_with_missing, axis=0)
df_train.shape

(144528, 25)

In [5]:
df_test = df_test.drop(['violation_zip_code', 'non_us_str_code', 'grafitti_status'], axis=1)

NameError: name 'df_test' is not defined

In [6]:
df_test.shape

NameError: name 'df_test' is not defined

In [5]:
cat_var = df_train.columns[df_train.dtypes == 'object']
len(cat_var)

14

In [6]:
num_var = df_train.columns[df_train.dtypes != 'object']
len(num_var)

11

In [19]:
df_train[num_var] = df_train[num_var].fillna(method='ffill')

In [20]:
df_train[cat_var] = df_train[cat_var].fillna(method='ffill')

In [31]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(solver='lbfgs', max_iter=500, random_state=17)

In [22]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [17]:
forest = RandomForestClassifier(n_estimators=100, random_state=17)

## Baseline only numeric var

In [35]:
df_no_cat, y = df_train.loc[:, df_train.dtypes != 'object'].drop('compliance', axis=1), df_train['compliance']

In [36]:
df_no_cat.columns

Index(['ticket_id', 'violation_street_number', 'mailing_address_str_number',
       'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount'],
      dtype='object')

In [37]:
df_no_cat_part, df_no_cat_valid, y_train_part, y_valid = train_test_split(df_no_cat, y, 
                                                                          test_size=.3, 
                                                                          stratify=y, 
                                                                          random_state=17)

In [38]:
np.mean(cross_val_score(forest, df_no_cat_part, y_train_part, cv=skf, scoring='roc_auc', error_score='raise'))

0.8011562034440189

In [39]:
forest.fit(df_no_cat_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [41]:
roc_auc_score(y_valid, forest.predict_proba(df_no_cat_valid)[:, 1])

0.8119952645993525

In [42]:
np.mean(cross_val_score(logit, df_no_cat_part, y_train_part, cv=skf, scoring='roc_auc', error_score='raise'))

0.753258521107564

In [43]:
logit.fit(df_no_cat_part, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [44]:
roc_auc_score(y_valid, logit.predict_proba(df_no_cat_valid)[:, 1])

0.7557382889704188

## LabelEncoder for cat var

In [55]:
from sklearn.preprocessing import LabelEncoder

In [56]:
label_encoder = LabelEncoder()

In [57]:
df_cat_label_enc = df_train.copy().drop('compliance', axis=1)
for col in df_train.columns[df_train.dtypes == 'object']:
    df_cat_label_enc[col] = label_encoder.fit_transform(df_cat_label_enc[col])

In [58]:
df_cat_label_enc.shape

(144528, 24)

In [59]:
df_cat_label_enc_part, df_cat_label_enc_valid = train_test_split(df_cat_label_enc, test_size=.3, 
                                                                 stratify=y, random_state=17)

In [60]:
np.mean(cross_val_score(forest, df_cat_label_enc_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8403463536693454

In [61]:
forest.fit(df_cat_label_enc_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [62]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_label_enc_valid)[:, 1])

0.8473507071348565

In [60]:
np.mean(cross_val_score(logit, df_cat_label_enc_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8403463536693454

In [61]:
logit.fit(df_cat_label_enc_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [62]:
roc_auc_score(y_valid, logit.predict_proba(df_cat_label_enc_valid)[:, 1])

0.8473507071348565

## Dummies, One Hot Encoding

In [12]:
df_train.columns[df_train.dtypes == 'object']

Index(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_name', 'mailing_address_str_name', 'city', 'state',
       'zip_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition'],
      dtype='object')

In [17]:
df_cat_dummies = pd.get_dummies(df_train, columns=[
       'agency_name', 'city']).drop('compliance', axis=1)

In [18]:
df_cat_dummies.shape

(144528, 3650)

In [20]:
y = df_train['compliance']

In [30]:
df_cat_dummies_part, df_cat_dummies_valid = train_test_split(df_cat_dummies, test_size=.3, 
                                                             stratify=y, random_state=17)

In [34]:
np.mean(cross_val_score(forest, df_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc', error_score='raise'))

MemoryError: 

In [ ]:
forest.fit(df_cat_dummies_part, y_train_part)

In [ ]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_dummies_valid)[:, 1])

## Mean target

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [25]:
df_train[num_var] = df_train[num_var].fillna(method='ffill')
df_train[cat_var] = df_train[cat_var].fillna(method='ffill')
train_df, y = df_train.copy(), df_train['compliance']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df.drop('compliance', axis=1), y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [26]:
def mean_target_enc(train_df, y_train, valid_df, skf):
    import warnings
    warnings.filterwarnings('ignore')
    
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()
    
    cat_features = train_df.columns[train_df.dtypes == 'object'].tolist()    

    for col in cat_features:
        new_train_df[col + '_mean_target'] = [glob_mean for _ in range(new_train_df.shape[0])]

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    
    new_train_df.drop(cat_features + ['y'], axis=1, inplace=True)
    
    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)['y'].mean())
        valid_df[col + '_mean_target'] = means.fillna(glob_mean)
        
    valid_df.drop(train_df.columns[train_df.dtypes == 'object'], axis=1, inplace=True)
    
    return new_train_df, valid_df

In [27]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [28]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.857843752435282

In [29]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [30]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.8607670590848804

In [32]:
np.mean(cross_val_score(logit, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.7533266973523431

In [33]:
logit.fit(train_mean_target_part, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [34]:
roc_auc_score(y_valid, logit.predict_proba(valid_mean_target)[:, 1])

0.7558031296442439

## Catboost

In [45]:
from catboost import CatBoostClassifier

In [46]:
ctb = CatBoostClassifier(random_seed=17)

In [48]:
train_df, y = df_train.drop('compliance', axis=1), df_train['compliance']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [49]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [50]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

Learning rate set to 0.058093
0:	learn: 0.6164224	total: 4.23s	remaining: 1h 10m 24s
1:	learn: 0.5548434	total: 6.63s	remaining: 55m 7s
2:	learn: 0.4998596	total: 10.9s	remaining: 1h 6s
3:	learn: 0.4554714	total: 14.8s	remaining: 1h 1m 28s
4:	learn: 0.4164401	total: 18.5s	remaining: 1h 1m 13s
5:	learn: 0.3847650	total: 22s	remaining: 1h 48s
6:	learn: 0.3574913	total: 25.4s	remaining: 59m 57s
7:	learn: 0.3338385	total: 28.8s	remaining: 59m 26s
8:	learn: 0.3153095	total: 31.5s	remaining: 57m 52s
9:	learn: 0.2981358	total: 35.4s	remaining: 58m 23s
10:	learn: 0.2851032	total: 37.4s	remaining: 56m 6s
11:	learn: 0.2733777	total: 43.5s	remaining: 59m 41s
12:	learn: 0.2624599	total: 47.4s	remaining: 59m 59s
13:	learn: 0.2555538	total: 50s	remaining: 58m 38s
14:	learn: 0.2473487	total: 53.7s	remaining: 58m 49s
15:	learn: 0.2409124	total: 57.4s	remaining: 58m 52s
16:	learn: 0.2355002	total: 1m	remaining: 58m 46s
17:	learn: 0.2316504	total: 1m 4s	remaining: 58m 52s
18:	learn: 0.2270822	total: 1m 

148:	learn: 0.1734392	total: 11m 23s	remaining: 1h 5m 2s
149:	learn: 0.1733765	total: 11m 27s	remaining: 1h 4m 56s
150:	learn: 0.1733627	total: 11m 32s	remaining: 1h 4m 51s
151:	learn: 0.1733356	total: 11m 36s	remaining: 1h 4m 43s
152:	learn: 0.1732930	total: 11m 40s	remaining: 1h 4m 37s
153:	learn: 0.1732580	total: 11m 44s	remaining: 1h 4m 31s
154:	learn: 0.1732044	total: 11m 48s	remaining: 1h 4m 24s
155:	learn: 0.1731597	total: 11m 53s	remaining: 1h 4m 17s
156:	learn: 0.1731022	total: 11m 57s	remaining: 1h 4m 10s
157:	learn: 0.1730695	total: 12m 1s	remaining: 1h 4m 4s
158:	learn: 0.1730199	total: 12m 5s	remaining: 1h 3m 58s
159:	learn: 0.1729766	total: 12m 9s	remaining: 1h 3m 52s
160:	learn: 0.1729594	total: 12m 14s	remaining: 1h 3m 45s
161:	learn: 0.1729183	total: 12m 18s	remaining: 1h 3m 37s
162:	learn: 0.1728919	total: 12m 22s	remaining: 1h 3m 33s
163:	learn: 0.1728701	total: 12m 26s	remaining: 1h 3m 26s
164:	learn: 0.1728490	total: 12m 31s	remaining: 1h 3m 21s
165:	learn: 0.17284

294:	learn: 0.1682461	total: 22m 59s	remaining: 54m 56s
295:	learn: 0.1682385	total: 23m 3s	remaining: 54m 50s
296:	learn: 0.1682229	total: 23m 7s	remaining: 54m 44s
297:	learn: 0.1682042	total: 23m 11s	remaining: 54m 38s
298:	learn: 0.1681427	total: 23m 15s	remaining: 54m 31s
299:	learn: 0.1681183	total: 23m 19s	remaining: 54m 25s
300:	learn: 0.1680586	total: 23m 23s	remaining: 54m 19s
301:	learn: 0.1680462	total: 23m 28s	remaining: 54m 14s
302:	learn: 0.1680133	total: 23m 32s	remaining: 54m 8s
303:	learn: 0.1679871	total: 23m 36s	remaining: 54m 2s
304:	learn: 0.1679461	total: 23m 40s	remaining: 53m 57s
305:	learn: 0.1679361	total: 23m 45s	remaining: 53m 52s
306:	learn: 0.1678924	total: 23m 49s	remaining: 53m 45s
307:	learn: 0.1678771	total: 23m 52s	remaining: 53m 39s
308:	learn: 0.1678620	total: 23m 57s	remaining: 53m 33s
309:	learn: 0.1678412	total: 24m 1s	remaining: 53m 28s
310:	learn: 0.1678090	total: 24m 5s	remaining: 53m 23s
311:	learn: 0.1677829	total: 24m 9s	remaining: 53m 15s

442:	learn: 0.1643775	total: 34m 20s	remaining: 43m 10s
443:	learn: 0.1643358	total: 34m 26s	remaining: 43m 7s
444:	learn: 0.1643304	total: 34m 30s	remaining: 43m 1s
445:	learn: 0.1643219	total: 34m 33s	remaining: 42m 56s
446:	learn: 0.1642903	total: 34m 38s	remaining: 42m 50s
447:	learn: 0.1642624	total: 34m 42s	remaining: 42m 46s
448:	learn: 0.1642431	total: 34m 47s	remaining: 42m 41s
449:	learn: 0.1642143	total: 34m 51s	remaining: 42m 36s
450:	learn: 0.1641871	total: 34m 56s	remaining: 42m 31s
451:	learn: 0.1641766	total: 34m 59s	remaining: 42m 25s
452:	learn: 0.1641556	total: 35m 3s	remaining: 42m 19s
453:	learn: 0.1641335	total: 35m 7s	remaining: 42m 14s
454:	learn: 0.1641114	total: 35m 11s	remaining: 42m 9s
455:	learn: 0.1640699	total: 35m 15s	remaining: 42m 3s
456:	learn: 0.1640296	total: 35m 18s	remaining: 41m 57s
457:	learn: 0.1640058	total: 35m 24s	remaining: 41m 54s
458:	learn: 0.1639837	total: 35m 29s	remaining: 41m 49s
459:	learn: 0.1639562	total: 35m 34s	remaining: 41m 45

590:	learn: 0.1607874	total: 45m 13s	remaining: 31m 17s
591:	learn: 0.1607775	total: 45m 17s	remaining: 31m 12s
592:	learn: 0.1607606	total: 45m 21s	remaining: 31m 8s
593:	learn: 0.1607531	total: 45m 26s	remaining: 31m 3s
594:	learn: 0.1607322	total: 45m 30s	remaining: 30m 58s
595:	learn: 0.1607001	total: 45m 34s	remaining: 30m 53s
596:	learn: 0.1606812	total: 45m 38s	remaining: 30m 48s
597:	learn: 0.1606626	total: 45m 43s	remaining: 30m 44s
598:	learn: 0.1606368	total: 45m 48s	remaining: 30m 39s
599:	learn: 0.1606247	total: 45m 52s	remaining: 30m 35s
600:	learn: 0.1605997	total: 45m 56s	remaining: 30m 30s
601:	learn: 0.1605867	total: 46m 1s	remaining: 30m 25s
602:	learn: 0.1605675	total: 46m 5s	remaining: 30m 20s
603:	learn: 0.1605539	total: 46m 9s	remaining: 30m 15s
604:	learn: 0.1605183	total: 46m 13s	remaining: 30m 10s
605:	learn: 0.1605064	total: 46m 17s	remaining: 30m 5s
606:	learn: 0.1604958	total: 46m 22s	remaining: 30m 1s
607:	learn: 0.1604811	total: 46m 26s	remaining: 29m 56s

...

289:	learn: 0.1690210	total: 19m 23s	remaining: 47m 27s
290:	learn: 0.1690098	total: 19m 27s	remaining: 47m 23s
291:	learn: 0.1689646	total: 19m 31s	remaining: 47m 20s
292:	learn: 0.1689350	total: 19m 35s	remaining: 47m 17s
293:	learn: 0.1689032	total: 19m 39s	remaining: 47m 13s
294:	learn: 0.1688746	total: 19m 43s	remaining: 47m 8s
295:	learn: 0.1688606	total: 19m 47s	remaining: 47m 4s
296:	learn: 0.1688354	total: 19m 51s	remaining: 47m
297:	learn: 0.1688184	total: 19m 55s	remaining: 46m 57s
298:	learn: 0.1687644	total: 20m	remaining: 46m 54s
299:	learn: 0.1687279	total: 20m 4s	remaining: 46m 50s
300:	learn: 0.1687083	total: 20m 7s	remaining: 46m 44s
301:	learn: 0.1686569	total: 20m 11s	remaining: 46m 40s
302:	learn: 0.1686067	total: 20m 18s	remaining: 46m 41s
303:	learn: 0.1685364	total: 20m 21s	remaining: 46m 37s
304:	learn: 0.1685024	total: 20m 29s	remaining: 46m 40s
305:	learn: 0.1684701	total: 20m 33s	remaining: 46m 38s
306:	learn: 0.1684557	total: 20m 38s	remaining: 46m 35s
307:

437:	learn: 0.1651354	total: 29m 39s	remaining: 38m 3s
438:	learn: 0.1651093	total: 29m 43s	remaining: 37m 59s
439:	learn: 0.1650940	total: 29m 47s	remaining: 37m 54s
440:	learn: 0.1650724	total: 29m 51s	remaining: 37m 50s
441:	learn: 0.1650361	total: 29m 55s	remaining: 37m 46s
442:	learn: 0.1650124	total: 29m 59s	remaining: 37m 42s
443:	learn: 0.1649838	total: 30m 4s	remaining: 37m 39s
444:	learn: 0.1649544	total: 30m 8s	remaining: 37m 35s
445:	learn: 0.1649416	total: 30m 12s	remaining: 37m 31s
446:	learn: 0.1649301	total: 30m 16s	remaining: 37m 27s
447:	learn: 0.1649112	total: 30m 20s	remaining: 37m 23s
448:	learn: 0.1648861	total: 30m 24s	remaining: 37m 19s
449:	learn: 0.1648607	total: 30m 28s	remaining: 37m 15s
450:	learn: 0.1648460	total: 30m 33s	remaining: 37m 11s
451:	learn: 0.1648137	total: 30m 37s	remaining: 37m 7s
452:	learn: 0.1647964	total: 30m 40s	remaining: 37m 2s
453:	learn: 0.1647811	total: 30m 45s	remaining: 36m 59s
454:	learn: 0.1647555	total: 30m 49s	remaining: 36m 5

585:	learn: 0.1619122	total: 39m 59s	remaining: 28m 14s
586:	learn: 0.1618936	total: 40m 3s	remaining: 28m 10s
587:	learn: 0.1618659	total: 40m 6s	remaining: 28m 6s
588:	learn: 0.1618550	total: 40m 10s	remaining: 28m 2s
589:	learn: 0.1618368	total: 40m 14s	remaining: 27m 58s
590:	learn: 0.1618070	total: 40m 18s	remaining: 27m 53s
591:	learn: 0.1617996	total: 40m 22s	remaining: 27m 49s
592:	learn: 0.1617764	total: 40m 26s	remaining: 27m 45s
593:	learn: 0.1617384	total: 40m 31s	remaining: 27m 41s
594:	learn: 0.1617198	total: 40m 35s	remaining: 27m 37s
595:	learn: 0.1617035	total: 40m 38s	remaining: 27m 33s
596:	learn: 0.1616849	total: 40m 42s	remaining: 27m 28s
597:	learn: 0.1616430	total: 40m 47s	remaining: 27m 25s
598:	learn: 0.1616369	total: 40m 52s	remaining: 27m 21s
599:	learn: 0.1616174	total: 40m 56s	remaining: 27m 17s
600:	learn: 0.1615961	total: 41m 1s	remaining: 27m 14s
601:	learn: 0.1615819	total: 41m 5s	remaining: 27m 9s
602:	learn: 0.1615541	total: 41m 8s	remaining: 27m 5s
6

733:	learn: 0.1586935	total: 50m 21s	remaining: 18m 14s
734:	learn: 0.1586755	total: 50m 24s	remaining: 18m 10s
735:	learn: 0.1586599	total: 50m 28s	remaining: 18m 6s
736:	learn: 0.1586202	total: 50m 32s	remaining: 18m 2s
737:	learn: 0.1586049	total: 50m 36s	remaining: 17m 57s
738:	learn: 0.1585771	total: 50m 40s	remaining: 17m 53s
739:	learn: 0.1585655	total: 50m 44s	remaining: 17m 49s
740:	learn: 0.1585425	total: 50m 48s	remaining: 17m 45s
741:	learn: 0.1585151	total: 50m 53s	remaining: 17m 41s
742:	learn: 0.1584782	total: 50m 57s	remaining: 17m 37s
743:	learn: 0.1584393	total: 51m	remaining: 17m 33s
744:	learn: 0.1584132	total: 51m 5s	remaining: 17m 29s
745:	learn: 0.1583911	total: 51m 9s	remaining: 17m 24s
746:	learn: 0.1583610	total: 51m 13s	remaining: 17m 21s
747:	learn: 0.1583406	total: 51m 17s	remaining: 17m 16s
748:	learn: 0.1583159	total: 51m 21s	remaining: 17m 12s
749:	learn: 0.1582945	total: 51m 25s	remaining: 17m 8s
750:	learn: 0.1582764	total: 51m 30s	remaining: 17m 4s
75

881:	learn: 0.1556167	total: 1h 1m 38s	remaining: 8m 14s
882:	learn: 0.1556075	total: 1h 1m 42s	remaining: 8m 10s
883:	learn: 0.1555947	total: 1h 1m 46s	remaining: 8m 6s
884:	learn: 0.1555756	total: 1h 1m 50s	remaining: 8m 2s
885:	learn: 0.1555419	total: 1h 1m 54s	remaining: 7m 57s
886:	learn: 0.1555082	total: 1h 1m 58s	remaining: 7m 53s
887:	learn: 0.1554890	total: 1h 2m 2s	remaining: 7m 49s
888:	learn: 0.1554733	total: 1h 2m 6s	remaining: 7m 45s
889:	learn: 0.1554553	total: 1h 2m 10s	remaining: 7m 41s
890:	learn: 0.1554396	total: 1h 2m 14s	remaining: 7m 36s
891:	learn: 0.1554167	total: 1h 2m 18s	remaining: 7m 32s
892:	learn: 0.1554018	total: 1h 2m 22s	remaining: 7m 28s
893:	learn: 0.1553715	total: 1h 2m 26s	remaining: 7m 24s
894:	learn: 0.1553575	total: 1h 2m 30s	remaining: 7m 20s
895:	learn: 0.1553521	total: 1h 2m 33s	remaining: 7m 15s
896:	learn: 0.1553456	total: 1h 2m 37s	remaining: 7m 11s
897:	learn: 0.1553128	total: 1h 2m 41s	remaining: 7m 7s
898:	learn: 0.1552955	total: 1h 2m 4

In [51]:
np.mean(cv_scores)

0.8863021184065765

In [52]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

Learning rate set to 0.06188
0:	learn: 0.6156819	total: 3.1s	remaining: 51m 32s
1:	learn: 0.5473088	total: 7.85s	remaining: 1h 5m 17s
2:	learn: 0.4902257	total: 12.8s	remaining: 1h 11m 5s
3:	learn: 0.4449368	total: 18.1s	remaining: 1h 15m 10s
4:	learn: 0.4044758	total: 23.2s	remaining: 1h 16m 53s
5:	learn: 0.3722014	total: 27.8s	remaining: 1h 16m 40s
6:	learn: 0.3461485	total: 32.2s	remaining: 1h 16m 10s
7:	learn: 0.3230645	total: 36s	remaining: 1h 14m 24s
8:	learn: 0.3036205	total: 40.8s	remaining: 1h 14m 49s
9:	learn: 0.2890316	total: 45.6s	remaining: 1h 15m 16s
10:	learn: 0.2759466	total: 50.3s	remaining: 1h 15m 25s
11:	learn: 0.2651074	total: 53.4s	remaining: 1h 13m 16s
12:	learn: 0.2577582	total: 54.9s	remaining: 1h 9m 25s
13:	learn: 0.2487368	total: 59.5s	remaining: 1h 9m 50s
14:	learn: 0.2412259	total: 1m 4s	remaining: 1h 10m 16s
15:	learn: 0.2345174	total: 1m 9s	remaining: 1h 10m 46s
16:	learn: 0.2290777	total: 1m 13s	remaining: 1h 11m 12s
17:	learn: 0.2260293	total: 1m 14s	rem

145:	learn: 0.1718085	total: 12m 31s	remaining: 1h 13m 16s
146:	learn: 0.1717852	total: 12m 36s	remaining: 1h 13m 12s
147:	learn: 0.1717330	total: 12m 42s	remaining: 1h 13m 6s
148:	learn: 0.1717226	total: 12m 46s	remaining: 1h 12m 56s
149:	learn: 0.1716942	total: 12m 50s	remaining: 1h 12m 48s
150:	learn: 0.1716814	total: 12m 56s	remaining: 1h 12m 43s
151:	learn: 0.1716257	total: 13m	remaining: 1h 12m 36s
152:	learn: 0.1715933	total: 13m 5s	remaining: 1h 12m 30s
153:	learn: 0.1714047	total: 13m 12s	remaining: 1h 12m 31s
154:	learn: 0.1713862	total: 13m 17s	remaining: 1h 12m 28s
155:	learn: 0.1713488	total: 13m 23s	remaining: 1h 12m 25s
156:	learn: 0.1712530	total: 13m 30s	remaining: 1h 12m 31s
157:	learn: 0.1712196	total: 13m 36s	remaining: 1h 12m 30s
158:	learn: 0.1711818	total: 13m 42s	remaining: 1h 12m 28s
159:	learn: 0.1711431	total: 13m 47s	remaining: 1h 12m 23s
160:	learn: 0.1710583	total: 13m 52s	remaining: 1h 12m 18s
161:	learn: 0.1710037	total: 13m 57s	remaining: 1h 12m 12s
162

287:	learn: 0.1668285	total: 25m 20s	remaining: 1h 2m 39s
288:	learn: 0.1667881	total: 25m 25s	remaining: 1h 2m 33s
289:	learn: 0.1667633	total: 25m 31s	remaining: 1h 2m 28s
290:	learn: 0.1666716	total: 25m 37s	remaining: 1h 2m 25s
291:	learn: 0.1666529	total: 25m 42s	remaining: 1h 2m 19s
292:	learn: 0.1666117	total: 25m 48s	remaining: 1h 2m 15s
293:	learn: 0.1665830	total: 25m 53s	remaining: 1h 2m 9s
294:	learn: 0.1665576	total: 25m 58s	remaining: 1h 2m 3s
295:	learn: 0.1664984	total: 26m 4s	remaining: 1h 2m 1s
296:	learn: 0.1664730	total: 26m 9s	remaining: 1h 1m 55s
297:	learn: 0.1664571	total: 26m 15s	remaining: 1h 1m 50s
298:	learn: 0.1664257	total: 26m 20s	remaining: 1h 1m 44s
299:	learn: 0.1664169	total: 26m 26s	remaining: 1h 1m 41s
300:	learn: 0.1664019	total: 26m 31s	remaining: 1h 1m 35s
301:	learn: 0.1663762	total: 26m 36s	remaining: 1h 1m 30s
302:	learn: 0.1663204	total: 26m 42s	remaining: 1h 1m 26s
303:	learn: 0.1663014	total: 26m 48s	remaining: 1h 1m 21s
304:	learn: 0.16628

435:	learn: 0.1630805	total: 38m 55s	remaining: 50m 21s
436:	learn: 0.1630626	total: 39m 1s	remaining: 50m 16s
437:	learn: 0.1630525	total: 39m 6s	remaining: 50m 10s
438:	learn: 0.1630456	total: 39m 11s	remaining: 50m 4s
439:	learn: 0.1629951	total: 39m 16s	remaining: 49m 58s
440:	learn: 0.1629770	total: 39m 22s	remaining: 49m 54s
441:	learn: 0.1629421	total: 39m 27s	remaining: 49m 48s
442:	learn: 0.1629146	total: 39m 32s	remaining: 49m 43s
443:	learn: 0.1629024	total: 39m 38s	remaining: 49m 37s
444:	learn: 0.1628821	total: 39m 43s	remaining: 49m 33s
445:	learn: 0.1628720	total: 39m 49s	remaining: 49m 27s
446:	learn: 0.1628634	total: 39m 54s	remaining: 49m 21s
447:	learn: 0.1628346	total: 39m 59s	remaining: 49m 16s
448:	learn: 0.1628170	total: 40m 5s	remaining: 49m 11s
449:	learn: 0.1628077	total: 40m 10s	remaining: 49m 6s
450:	learn: 0.1627641	total: 40m 16s	remaining: 49m 1s
451:	learn: 0.1627080	total: 40m 21s	remaining: 48m 55s
452:	learn: 0.1626974	total: 40m 26s	remaining: 48m 49

583:	learn: 0.1601231	total: 51m 54s	remaining: 36m 58s
584:	learn: 0.1601033	total: 52m	remaining: 36m 53s
585:	learn: 0.1600790	total: 52m 5s	remaining: 36m 47s
586:	learn: 0.1600737	total: 52m 10s	remaining: 36m 42s
587:	learn: 0.1600499	total: 52m 16s	remaining: 36m 37s
588:	learn: 0.1600342	total: 52m 21s	remaining: 36m 32s
589:	learn: 0.1600112	total: 52m 26s	remaining: 36m 26s
590:	learn: 0.1599908	total: 52m 31s	remaining: 36m 21s
591:	learn: 0.1599738	total: 52m 36s	remaining: 36m 15s
592:	learn: 0.1599591	total: 52m 42s	remaining: 36m 10s
593:	learn: 0.1599511	total: 52m 47s	remaining: 36m 5s
594:	learn: 0.1599360	total: 52m 53s	remaining: 36m
595:	learn: 0.1599190	total: 52m 58s	remaining: 35m 54s
596:	learn: 0.1598819	total: 53m 4s	remaining: 35m 49s
597:	learn: 0.1598765	total: 53m 9s	remaining: 35m 44s
598:	learn: 0.1598541	total: 53m 15s	remaining: 35m 39s
599:	learn: 0.1598468	total: 53m 20s	remaining: 35m 33s
600:	learn: 0.1598282	total: 53m 26s	remaining: 35m 28s
601:

730:	learn: 0.1573478	total: 1h 5m 5s	remaining: 23m 57s
731:	learn: 0.1573388	total: 1h 5m 10s	remaining: 23m 51s
732:	learn: 0.1573207	total: 1h 5m 16s	remaining: 23m 46s
733:	learn: 0.1572952	total: 1h 5m 22s	remaining: 23m 41s
734:	learn: 0.1572881	total: 1h 5m 27s	remaining: 23m 36s
735:	learn: 0.1572655	total: 1h 5m 32s	remaining: 23m 30s
736:	learn: 0.1572428	total: 1h 5m 37s	remaining: 23m 25s
737:	learn: 0.1572196	total: 1h 5m 42s	remaining: 23m 19s
738:	learn: 0.1572101	total: 1h 5m 47s	remaining: 23m 14s
739:	learn: 0.1571853	total: 1h 5m 54s	remaining: 23m 9s
740:	learn: 0.1571696	total: 1h 5m 59s	remaining: 23m 3s
741:	learn: 0.1571562	total: 1h 6m 3s	remaining: 22m 58s
742:	learn: 0.1571379	total: 1h 6m 9s	remaining: 22m 52s
743:	learn: 0.1571196	total: 1h 6m 14s	remaining: 22m 47s
744:	learn: 0.1571144	total: 1h 6m 18s	remaining: 22m 41s
745:	learn: 0.1570957	total: 1h 6m 24s	remaining: 22m 36s
746:	learn: 0.1570878	total: 1h 6m 29s	remaining: 22m 31s
747:	learn: 0.15705

871:	learn: 0.1548315	total: 1h 17m 45s	remaining: 11m 24s
872:	learn: 0.1548186	total: 1h 17m 50s	remaining: 11m 19s
873:	learn: 0.1548102	total: 1h 17m 56s	remaining: 11m 14s
874:	learn: 0.1547862	total: 1h 18m 1s	remaining: 11m 8s
875:	learn: 0.1547729	total: 1h 18m 7s	remaining: 11m 3s
876:	learn: 0.1547585	total: 1h 18m 12s	remaining: 10m 58s
877:	learn: 0.1547416	total: 1h 18m 18s	remaining: 10m 52s
878:	learn: 0.1547323	total: 1h 18m 24s	remaining: 10m 47s
879:	learn: 0.1547160	total: 1h 18m 29s	remaining: 10m 42s
880:	learn: 0.1547067	total: 1h 18m 35s	remaining: 10m 36s
881:	learn: 0.1546921	total: 1h 18m 39s	remaining: 10m 31s
882:	learn: 0.1546704	total: 1h 18m 46s	remaining: 10m 26s
883:	learn: 0.1546569	total: 1h 18m 54s	remaining: 10m 21s
884:	learn: 0.1546277	total: 1h 19m 4s	remaining: 10m 16s
885:	learn: 0.1546086	total: 1h 19m 13s	remaining: 10m 11s
886:	learn: 0.1545910	total: 1h 19m 20s	remaining: 10m 6s
887:	learn: 0.1545797	total: 1h 19m 26s	remaining: 10m 1s
888:

In [53]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.892485851208494